#  Package preparation

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
!pip3 install torch==1.5.1+cu101 torchvision==0.6.1+cu101 -f https://download.pytorch.org/whl/torch_stable.html

Looking in links: https://download.pytorch.org/whl/torch_stable.html
     |████████████████████████████████| 704.4MB 25kB/s 
     |████████████████████████████████| 6.6MB 15.7MB/s 
  Found existing installation: torch 1.7.0+cu101
    Uninstalling torch-1.7.0+cu101:
      Successfully uninstalled torch-1.7.0+cu101
  Found existing installation: torchvision 0.8.1+cu101
    Uninstalling torchvision-0.8.1+cu101:
      Successfully uninstalled torchvision-0.8.1+cu101


# Environment setup

In [ ]:
# Standard libraries
import os
import pandas as pd
import time
import sys
from os.path import exists, join, basename, splitext
import glob
import numpy as np
import shutil
import matplotlib.pyplot as plt
import cv2
from ast import literal_eval as make_tuple
from tqdm import tqdm
from joblib import Parallel, delayed 
import codecs
import argparse
import unicodedata
import editdistance

# Torch
import torch
import torch.backends.cudnn as cudnn
import torch.nn.init as init
import torch.optim as optim
import torch.utils.data
import torch.nn.functional as F
print(torch.cuda.is_available())


True


In [ ]:
%cd /content/
git_repo_url = 'https://github.com/jandziak/deep-text-recognition-benchmark.git'
project_name = splitext(basename(git_repo_url))[0]
project_name = project_name + '-pol'
if not exists(project_name):
  !git clone -q {git_repo_url} deep-text-recognition-benchmark-pol
  !cd {project_name}
  

sys.path.append(project_name)
from utils import CTCLabelConverter, CTCLabelConverterForBaiduWarpctc, AttnLabelConverter, Averager
from dataset import hierarchical_dataset, AlignCollate, Batch_Balanced_Dataset,RawDataset
from model import Model

/content


In [ ]:
%cd /content/
git_repo_url = 'https://github.com/githubharald/CTCWordBeamSearch.git'
project_name = splitext(basename(git_repo_url))[0]

if not exists(project_name):
  !git clone -q {git_repo_url}
  %cd {project_name}
  !pip install .

from word_beam_search import WordBeamSearch

def wbs_decode(feedMat, corpus, chars, wordChars):
    "decode using word beam search. Result is tuple, first entry is label string, second entry is char string."

    # decode using the "Words" mode of word beam search with beam width set to 25 and add-k smoothing to 0.0
    assert len(chars) + 1 == feedMat.shape[2]
    wbs = WordBeamSearch(25, 'Words', 0.0, corpus.encode('utf8'), chars.encode('utf8'), wordChars.encode('utf8'))
    res = wbs.compute(feedMat)

    # result is string of labels terminated by blank (similar to C-strings) if shorter than T
    blank = len(chars)
    s = ''

    for label in res[0]:
        if label == blank:
            break
        s += chars[label]  # map label to char
    return res[0], s

/content
/content/CTCWordBeamSearch
Processing /content/CTCWordBeamSearch
  Created wheel for word-beam-search: filename=word_beam_search-1.0.0-cp36-cp36m-linux_x86_64.whl size=1177491 sha256=3543eec9800c3f1fa694b326e9dc9aa8f3faff6388272dda743a9546a29238b5
  Stored in directory: /tmp/pip-ephem-wheel-cache-q8z03_bj/wheels/a9/69/4c/9d6acbecc7bf4b47c5072b213d9b08e4b9c43864bbed5206cc
Successfully built word-beam-search


In [ ]:
def cut_top_part_image(image_name, input_folder, output_folder, cut_top): 
    in_path = input_folder + image_name
    out_path = output_folder + image_name
    image = cv2.imread(in_path)
    cv2.imwrite(out_path, image[:cut_top,:])

In [ ]:
def create_full_corpus(words):
  return ' '.join(words.word.values)

def create_corpus(filename, words, link_dict): 
  filename = filename.replace('__', '_')
  filename = '_'.join(filename.split('_')[:-1])
  res = link_dict[filename]
  res = [i for i, v in enumerate(list(words.word.values)) if v == res[0] or v == res[1]]
  if len(res) == 1:
    corpus = words.word.values[res[0]]
  else: 
    corpus = ' '.join(words.word.values[res[0]:res[1]+2])
  return corpus

In [ ]:
def warpBox(image,
            box,
            target_height=None,
            target_width=None,
            margin=0,
            cval=None,
            return_transform=False,
            skip_rotate=False):
    """Warp a boxed region in an image given by a set of four points into
    a rectangle with a specified width and height. Useful for taking crops
    of distorted or rotated text.
    Args:
        image: The image from which to take the box
        box: A list of four points starting in the top left
            corner and moving clockwise.
        target_height: The height of the output rectangle
        target_width: The width of the output rectangle
        return_transform: Whether to return the transformation
            matrix with the image.
    """
    if cval is None:
        cval = (0, 0, 0) if len(image.shape) == 3 else 0
    if not skip_rotate:
        box, _ = keras_ocr.tools.get_rotated_box(box)
    w, h = keras_ocr.tools.get_rotated_width_height(box)
    margin = h*margin
    assert (
        (target_width is None and target_height is None)
        or (target_width is not None and target_height is not None)), \
            'Either both or neither of target width and height must be provided.'
    if target_width is None and target_height is None:
        target_width = w
        target_height = h
    scale = min(target_width / w, target_height / h)
    M = cv2.getPerspectiveTransform(src=box,
                                    dst=np.array([[margin, margin], [scale * w - margin, margin],
                                                  [scale * w - margin, scale * h - margin],
                                                  [margin, scale * h - margin]]).astype('float32'))
    crop = cv2.warpPerspective(image, M, dsize=(int(scale * w), int(scale * h)))
    target_shape = (target_height, target_width, 3) if len(image.shape) == 3 else (target_height,
                                                                                   target_width)
    full = (np.zeros(target_shape) + cval).astype('uint8')
    full[:crop.shape[0], :crop.shape[1]] = crop
    if return_transform:
        return full, M
    return full
def calc_box(img_path, detector):
    low_text_options = [0.3, 0.2, 0.1, 0.05, 0.01, 0.001, 0]
    low_text_index = 0
    img = keras_ocr.tools.read(img_path)
    image = keras_ocr.detection.compute_input(img)
    bboxes = keras_ocr.detection.getBoxes(detector.model.predict(np.array([image])), 
                                         text_threshold=0.9)

    while bboxes[0].shape[0]>1:
        bboxes = keras_ocr.detection.getBoxes(detector.model.predict(np.array([image])), 
                                             text_threshold=low_text_options[low_text_index])
        low_text_index +=1
    return bboxes, img

In [ ]:
def prepare_corpus_tools(links_path, words_path): 
  ''' Function preparating the word lists and word ranges for constrained wbs'''
  links = pd.read_csv(links_path)
  words = pd.read_csv(words_path)
  links = links[['0','1','2','3']]
  links.columns = ['link_part', 'range', 'start', 'end']
  link_dict ={}
  for i in range(len(links)):
    link_dict[links.iloc[i]['link_part']] = (links.iloc[i]['start'],
                                             links.iloc[i]['end'] )
  return link_dict, words

def parse_model_arguments(model_path, character):
  ''' Function that parses simple model arguments for the inference '''
  parser = argparse.Namespace()
  parser.eval_data=True
  parser.benchmark_all_eval=False
  parser.workers=4
  parser.batch_size=1
  parser.saved_model=model_path
  parser.batch_max_length=1
  parser.imgH=32
  parser.imgW=100
  parser.rgb=False
  parser.character=character
  parser.sensitive=False
  parser.PAD=False
  parser.Transformation='TPS'
  parser.FeatureExtraction='ResNet'
  parser.SequenceModeling='BiLSTM'
  parser.Prediction='CTC'
  parser.num_fiducial=20
  parser.input_channel=1
  parser.output_channel=512
  parser.hidden_size=256
  parser.num_gpu=0

  opt = parser

  opt.num_gpu = torch.cuda.device_count()
  return opt

def create_model(opt, device, converter):
  opt.num_class = len(converter.character)


  model = Model(opt)
  model = torch.nn.DataParallel(model).to(device)

  model.load_state_dict(torch.load(opt.saved_model, map_location=torch.device('cuda')))
  model.eval()
  print("model ready")
  return model

def create_dataloader(data_path, opt):
  AlignCollate_demo = AlignCollate(imgH=opt.imgH, imgW=opt.imgW, keep_ratio_with_pad=opt.PAD)
  demo_data = RawDataset(root=data_path, opt=opt)  # use RawDataset
  demo_loader = torch.utils.data.DataLoader(
      demo_data, batch_size=opt.batch_size,
      shuffle=False,
      num_workers=int(opt.workers),
      collate_fn=AlignCollate_demo, pin_memory=True)
  return demo_loader

def decode_prediction(preds, preds_str, image_path_list, words, link_dict, 
                      corpus_full, character):
  preds_np = preds.cpu().numpy()
  x = np.array([np.array([np.append(x[1:], x[0])]) for x in preds_np[0]])
  image_name = image_path_list[0].split('/')[-1]
  corpus = create_corpus(image_name, words, link_dict)

  res1 = wbs_decode(x, corpus, character, character)
  res2 = wbs_decode(x, corpus_full, character, character)

  temp_res = (f'{image_name}', res1[1], res2[1], preds_str[0], round(float(confidence_score.cpu().numpy()),4))

  return temp_res
  
def predict(image, model, batch_size, device, opt, converter):
  length_for_pred = torch.IntTensor([opt.batch_max_length] * batch_size).to(device)
  text_for_pred = torch.LongTensor(batch_size, opt.batch_max_length + 1).fill_(0).to(device)
  preds = model(image, text_for_pred)
  preds_prob = F.softmax(preds, dim=2)
  preds_max_prob, _ = preds_prob.max(dim=2)
  confidence_score = preds_max_prob[0].cumprod(dim=0)[-1]
  preds_size = torch.IntTensor([preds.size(1)] * batch_size)
  _, preds_index = preds.max(2)
  preds_str = converter.decode(preds_index, preds_size)
  return preds, preds_str, confidence_score

In [ ]:

def calculate_levenshtein_score(s: str, t: str) -> float:
    return editdistance.eval(s, t) / max(len(s), len(t))

def eval_predictions(act, pred):
    match = [x==y for x,y in zip(act, pred)]
    accuracy = sum(match)/len(match)

    edit = [editdistance.eval(x, y) for x,y in zip(act, pred)]
    edit_distance = sum(edit)/len(edit)

    normalised_edit = [calculate_levenshtein_score(x, y) for x,y in zip(act, 
                                                                        pred)]
    normalised_edit_distance = sum(normalised_edit)/len(normalised_edit)                                                                  
    
    return accuracy, normalised_edit, normalised_edit_distance

# Pipeline parameters

In [ ]:
character="0123456789abcdefghijklmnoprstuwyzóąćęłńśźż"
cudnn.benchmark = True
cudnn.deterministic = True
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu') 

#Preparation of the data

In [ ]:
%cd /content
!gdown --id 1GWlf5bVi0pCM8Ixyqza6TJlMtA836eq9
!gdown --id 1PFKI761GLTuZYFV2vGuv4MAblNJQlKzI

link_dict, words = prepare_corpus_tools(links_path='/content/link_to_range100k.csv',
                                         words_path='/content/words_in_order.csv')

/content
Downloading...
From: https://drive.google.com/uc?id=1GWlf5bVi0pCM8Ixyqza6TJlMtA836eq9
To: /content/link_to_range100k.csv
100% 43.8k/43.8k [00:00<00:00, 33.3MB/s]
Downloading...
From: https://drive.google.com/uc?id=1PFKI761GLTuZYFV2vGuv4MAblNJQlKzI
To: /content/words_in_order.csv
100% 1.62M/1.62M [00:00<00:00, 103MB/s]


# Prepare model

In [ ]:
!gdown --id 1FocnxQzFBIjDT2F9BkNUiLdo1cC3eaO0
!gdown --id 1o8GYwkDWJ_hE76C9z4iyh0GfBLf45jTK

Downloading...
From: https://drive.google.com/uc?id=1FocnxQzFBIjDT2F9BkNUiLdo1cC3eaO0
To: /content/TPS-ResNet-BiLSTM-CTC.pth
196MB [00:01, 125MB/s]
Downloading...
From: https://drive.google.com/uc?id=1o8GYwkDWJ_hE76C9z4iyh0GfBLf45jTK
To: /content/best_accuracy.pth
196MB [00:02, 80.7MB/s]


In [ ]:
opt = parse_model_arguments(model_path='/content/best_accuracy.pth', 
                      character=character)
converter = CTCLabelConverter(opt.character)
model = create_model(opt, device, converter)

model ready


In [ ]:
%cd /content
!mkdir data
!gdown --id 1Fz7MdKwV1MtL6Eh5HUNh9lAoVW1O5CqA 
!gdown --id 1IyoszVzyR9GzVQAGobcuR2EDMzZPv309
!unzip -q new_dataset.zip -d data

data = pd.read_csv('predictions_output_ver2.csv')
data['file_name_nfd'] = [unicodedata.normalize('NFD', file_name) \
  for file_name in data['file_name']]
# Sanity check to see if we can map files from the system to the results in the 
# pandas Data Frame
# len([file_name for file_name in os.listdir('data/new_dataset') \
#     if file_name in data['file_name_nfd'].values])
#data.head()

/content
mkdir: cannot create directory ‘data’: File exists
Downloading...
From: https://drive.google.com/uc?id=1Fz7MdKwV1MtL6Eh5HUNh9lAoVW1O5CqA
To: /content/predictions_output_ver2.csv
2.72MB [00:00, 87.8MB/s]
Downloading...
From: https://drive.google.com/uc?id=1IyoszVzyR9GzVQAGobcuR2EDMzZPv309
To: /content/new_dataset.zip
288MB [00:01, 166MB/s]
replace data/new_dataset/HistJezXVIIw_Ten10_0279.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: N


In [ ]:
%cd /content
corpus_full = create_full_corpus(words)
resss= []
errors_list = {}
demo_loader = create_dataloader(data_path='data/new_dataset/', opt=opt)

with torch.no_grad():
  for image_tensors, image_path_list in tqdm(demo_loader):
    batch_size = image_tensors.size(0)
    image = image_tensors.to(device)
    try:

      # predict
      preds, preds_str, confidence_score = predict(image, model, batch_size, 
                                                   device, opt, converter)
      # postprocess
      temp_res = decode_prediction(preds, preds_str, image_path_list, words, link_dict, 
                                   corpus_full, character )
      resss.append(temp_res)

    except: 
      errors_list[image_path_list[0]] = 1
    
#resss

/content


100%|██████████| 19999/19999 [49:01<00:00,  6.80it/s]


In [ ]:
)resss = pd.DataFrame(resss)
resss.columns = ['file_name', 'wbs', 'wbs_full', 'model', 'conf']

In [ ]:
!gdown --id 1ZoMRbML-4x9d5_uonQ3DuwGoBqsvuuoH

In [ ]:
data = pd.read_csv('17th_century_labels.csv')
data['file_name_nfd'] = [unicodedata.normalize('NFD', file_name) \
  for file_name in data['file_name']]
data.merge(resss, on='file_name', how = 'left')
eval_predictions(data.label, data.wbs)

In [ ]:
eval_predictions(data.label, data.wbs_full)

In [ ]:
eval_predictions(data.label, data.model)